In [2]:
import torch
from sklearn.base import BaseEstimator
from typing import TypedDict
import numpy as np
import numpy
from sklearn.base import clone
from sklearn.model_selection import GridSearchCV

USER_FUNCTIONS = {
    'sum': lambda origin_features, updated_features, sum_neighbors, mul_neighbors, num_neighbors: sum_neighbors,
    'mean': lambda origin_features, updated_features, sum_neighbors, mul_neighbors, num_neighbors: sum_neighbors / num_neighbors,
    'diff_of_origin_mean': lambda origin_features, updated_features, sum_neighbors, mul_neighbors, num_neighbors: origin_features - sum_neighbors / num_neighbors,
    'diff_of_updated_mean': lambda origin_features, updated_features, sum_neighbors, mul_neighbors, num_neighbors: updated_features - sum_neighbors / num_neighbors,
    'sum_of_origin_mean': lambda origin_features, updated_features, sum_neighbors, mul_neighbors, num_neighbors: origin_features + sum_neighbors / num_neighbors,
    'sum_of_updated_mean': lambda origin_features, updated_features, sum_neighbors, mul_neighbors, num_neighbors: updated_features + sum_neighbors / num_neighbors,
}
## Assumption: the overall prediction perf improved when the performance of inidividual predictiors improves
##TODO More input_validation, grid search method whoch accepts the same params
class Framework:    
    
    def __init__(self, user_functions, 
                 hops_list:list[int],
                 clfs:list,
                 gpu_idx:int|None=None,
                 handle_nan:float|None=None,
                attention_configs:list=[]) -> None:
        self.user_functions = user_functions
        self.hops_list:list[int] = hops_list
        self.clfs:list[int] = clfs
        self.trained_clfs = None
        self.gpu_idx:int|None = gpu_idx
        self.handle_nan:float|int|None = handle_nan
        self.attention_configs = attention_configs
        self.device:torch.DeviceObjType = torch.device(f"cuda:{str(self.gpu_idx)}") if self.gpu_idx is not None and torch.cuda.is_available() else torch.device("cpu")
    
    def update_user_function(self):
        if self.user_function in USER_FUNCTIONS:
            self.user_function = USER_FUNCTIONS[self.user_function]
        else:
            raise Exception(f"Only the following string values are valid inputs for the user function: {[key for key in USER_FUNCTIONS]}. You can also specify your own function for aggregatioon.")
            
    def get_features(self,
                     X:torch.FloatTensor|numpy._typing.NDArray,
                     edge_index:torch.LongTensor|numpy._typing.NDArray,
                     mask:torch.BoolTensor|numpy._typing.NDArray,
                    is_training:bool = False) -> tuple[torch.FloatTensor, torch.FloatTensor]:
        if mask is None:
            mask = torch.ones(X.shape[0]).type(torch.bool)
#         if isinstance(self.user_function, str):
#             self.update_user_function()
        ## To tensor
        X = Framework.get_feature_tensor(X)
        edge_index = Framework.get_edge_index_tensor(edge_index)
        mask = Framework.get_mask_tensor(mask)
        
        ## To device
        X = self.shift_tensor_to_device(X)
        edge_index = self.shift_tensor_to_device(edge_index)
        mask = self.shift_tensor_to_device(mask)
        
        aggregated_train_features_list = []
        ## Aggregate
        for hop_idx in range(len(self.hops_list)):
            neighbor_features = self.aggregate(X, edge_index, hop_idx, is_training)
            aggregated_train_features_list.append(neighbor_features[mask])
        
        return aggregated_train_features_list
    
    def aggregate(self, X:torch.FloatTensor, edge_index:torch.LongTensor,hop_idx, is_training:bool=False) -> torch.FloatTensor: 
        original_features = X
        features_for_aggregation:torch.FloatTensor = torch.clone(X)
        hops_list = self.hops_list[hop_idx]
        for i, hop in enumerate(range(hops_list)):
            if self.attention_configs[hop_idx] and self.attention_configs[hop_idx]["inter_layer_normalize"]:
                features_for_aggregation = torch.nn.functional.normalize(features_for_aggregation, dim = 0)
            source_lift = features_for_aggregation.index_select(0, edge_index[0])
            target = edge_index[1]
            
            if self.attention_configs[hop_idx] and self.attention_configs[hop_idx]["use_pseudo_attention"]:
                source_lift = self.apply_attention_mechanism(source_lift, features_for_aggregation, target,self.attention_configs[hop_idx], is_training)
            
            summed_neighbors = torch.zeros_like(features_for_aggregation, device=self.device).scatter_reduce(0, target.unsqueeze(0).repeat(features_for_aggregation.shape[1], 1).t(), source_lift, reduce="sum", include_self = False)
            summed_neighbors = torch.zeros_like(features_for_aggregation, device=self.device).scatter_(0, target.unsqueeze(0).repeat(features_for_aggregation.shape[1], 1).t(), source_lift, reduce="add")
            multiplied_neighbors = torch.ones_like(features_for_aggregation, device=self.device).scatter_reduce(0, target.unsqueeze(0).repeat(features_for_aggregation.shape[1], 1).t(), source_lift, reduce="prod", include_self = False)
            mean_neighbors = torch.zeros_like(features_for_aggregation, device=self.device).scatter_reduce(0, target.unsqueeze(0).repeat(features_for_aggregation.shape[1], 1).t(), source_lift, reduce="mean", include_self = False)
            max_neighbors = torch.zeros_like(features_for_aggregation, device=self.device).scatter_reduce(0, target.unsqueeze(0).repeat(features_for_aggregation.shape[1], 1).t(), source_lift, reduce="amax", include_self = False)
            min_neighbors = torch.zeros_like(features_for_aggregation, device=self.device).scatter_reduce(0, target.unsqueeze(0).repeat(features_for_aggregation.shape[1], 1).t(), source_lift, reduce="amin", include_self = False)

            num_source_neighbors = torch.zeros(features_for_aggregation.shape[0], dtype=torch.float, device=self.device)
            num_source_neighbors.scatter_reduce(0, target, torch.ones_like(target, dtype=torch.float, device=self.device), reduce="sum", include_self = False)
            num_source_neighbors = num_source_neighbors.unsqueeze(-1)

            user_function = self.user_functions[hop_idx]
            updated_features = features_for_aggregation ## just renaming so that the key in the user function is clear
            user_function_kwargs = {
                                'original_features':original_features,
                                'updated_features':updated_features,
                                'summed_neighbors':summed_neighbors,
                                'multiplied_neighbors':multiplied_neighbors,
                                'mean_neighbors':mean_neighbors,
                                'max_neighbors':max_neighbors,
                                'min_neighbors':min_neighbors,
                                'num_source_neighbors':num_source_neighbors,
                                'hop':hop}
            out = user_function(user_function_kwargs)
            
            if self.handle_nan is not None:
                out = torch.nan_to_num(out, nan=self.handle_nan)
            features_for_aggregation = out
        return features_for_aggregation
    
    def apply_attention_mechanism(self, source_lift:torch.FloatTensor,
                                  features_for_aggregation:torch.FloatTensor,
                                  target:torch.LongTensor,
                                  attention_config,
                                 is_training:bool = False) -> torch.FloatTensor:
        cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
        score = cos(source_lift, features_for_aggregation.index_select(0, target))
        dropout_tens = None
        
        origin_scores = torch.clone(score)
        if attention_config["cosine_eps"]:
            score[score < attention_config["cosine_eps"]] = -torch.inf
        if attention_config["dropout_attn"] is not None and is_training:
            dropout_tens = torch.FloatTensor(score.shape[0]).uniform_(0, 1)
            score[dropout_tens < attention_config["dropout_attn"]] = -torch.inf
        exp_score = torch.exp(score)
        summed_exp_score = torch.zeros_like(exp_score).scatter(0, target,exp_score, reduce="add")
        target_lifted_summed_exp_score = summed_exp_score.index_select(0, target)
        normalized_scores = exp_score / target_lifted_summed_exp_score
        source_lift = normalized_scores.unsqueeze(1) * source_lift
        return source_lift
    
    def fit(self,
            X_train:torch.FloatTensor|numpy._typing.NDArray,
            edge_index:torch.LongTensor|numpy._typing.NDArray,
            y_train:torch.LongTensor|numpy._typing.NDArray,
            train_mask:torch.BoolTensor|numpy._typing.NDArray|None,
            kwargs_list = None
            ) -> BaseEstimator:   
        if train_mask is None:
            train_mask = torch.ones(X_train.shape[0]).type(torch.bool)
            
        y_train = Framework.get_label_tensor(y_train)
        y_train = y_train[train_mask]
        
        self.validate_input()
        
        aggregated_train_features_list = self.get_features(X_train, edge_index, train_mask, True)  
        
        trained_clfs = []
        for i, aggregated_train_features in enumerate(aggregated_train_features_list):
            clf = clone(self.clfs[i])
            kwargs = kwargs_list[i] if kwargs_list and len(kwargs_list)>i is not None else {}
            clf.fit(aggregated_train_features.cpu().numpy(), y_train,**kwargs)
            trained_clfs.append(clf)
        self.trained_clfs = trained_clfs
        return trained_clfs    
    
    def predict_proba(self, X_test:torch.FloatTensor|numpy._typing.NDArray,
                      edge_index:torch.LongTensor|numpy._typing.NDArray,
                      test_mask:torch.BoolTensor|numpy._typing.NDArray|None,
                      weights=None,
                     kwargs_list = None):  
        if test_mask is None:
            test_mask = torch.ones(X_test.shape[0]).type(torch.bool)
        aggregated_test_features_list = self.get_features(X_test, edge_index, test_mask)
        
        pred_probas = []
        for i, clf in enumerate(self.trained_clfs):
            aggregated_test_features = aggregated_test_features_list[i]
            kwargs = kwargs_list[i] if kwargs_list is not None else {}
            pred_proba = clf.predict_proba(aggregated_test_features.cpu().numpy(),**kwargs) if kwargs else clf.predict_proba(aggregated_test_features.cpu().numpy())
            pred_probas.append(pred_proba)
        final_pred_proba = np.average(np.asarray(pred_probas), weights=weights, axis=0)
        return final_pred_proba
        
    
    def predict(self,
                X_test:torch.FloatTensor|numpy._typing.NDArray,
                edge_index:torch.LongTensor|numpy._typing.NDArray,
                test_mask:torch.BoolTensor|numpy._typing.NDArray|None,
                 weights=None,
                     kwargs_list = None):
        return self.predict_proba(X_test, edge_index, test_mask, weights, kwargs_list).argmax(1)
        

    def validate_input(self):
        pass
            
    @staticmethod
    def get_feature_tensor(X:torch.FloatTensor|numpy._typing.NDArray) -> torch.FloatTensor|None:
        if not torch.is_tensor(X):
            try:
                return torch.from_numpy(X).type(torch.float)
            except:
                raise Exception("Features input X must be numpy array or torch tensor!")
                return None 
        return X
    
    @staticmethod
    def get_label_tensor(y:torch.LongTensor|numpy._typing.NDArray) -> torch.LongTensor|None:
        if not torch.is_tensor(y):
            try:
                return torch.from_numpy(y).type(torch.long)
            except:
                raise Exception("Label input y must be numpy array or torch tensor!")
                return None
        return y
    
    @staticmethod
    def get_mask_tensor(mask:torch.BoolTensor|numpy._typing.NDArray) -> torch.BoolTensor|None:
        if not torch.is_tensor(mask):
            try:
                return torch.from_numpy(mask).type(torch.bool)
            except:
                raise Exception("Input mask must be numpy array or torch tensor!")
                return None
        return mask
            
    @staticmethod
    def get_edge_index_tensor(edge_index:torch.LongTensor|numpy._typing.NDArray) -> torch.LongTensor|None:
        if not torch.is_tensor(edge_index):
            try:
                edge_index =  torch.from_numpy(edge_index).type(torch.long)
                Framework.validate_edge_index(edge_index)
                return edge_index
            except:
                raise Exception("Edge index must be numpy array or torch tensor")
                return None
        return edge_index
    
    @staticmethod
    def validate_edge_index(edge_index:torch.LongTensor) -> None:
        if edge_index.shape[0] != 2:
            raise Exception("Edge index must have the shape 2 x NumberOfEdges")
            # TODO: check max edge index and shape of features
    
    def shift_tensor_to_device(self,
                               t:torch.FloatTensor) -> torch.FloatTensor:
        if self.gpu_idx is not None:
            return t.to(self.device) 
        return t
    
    def validate_grid_input(self, grid_params):
        if len(grid_params) != 1 and self.use_feature_based_aggregation:
            raise Exception("You need to provide grid parameter for the classifier!")
        if len(grid_params) != 2 and not self.use_feature_based_aggregation:
            raise Exception("You need to provide two grid parameter, one for each classifier!")
        return
    
    def hyper_param_tuning(spaces, objectives, n_iter, X_train, y_train, X_val, y_val):
        ## bayes optim
        pass

In [47]:
from xgboost import XGBClassifier
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.utils import add_self_loops
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC

dataset = Planetoid(root='/tmp/Cora', name='Cora', split="public")
dataset.transform = T.NormalizeFeatures()

X =  dataset[0].x 
y =  dataset[0].y 

test =  dataset[0].test_mask
train = dataset[0].train_mask 
val =  dataset[0].val_mask

edge_index = dataset[0].edge_index 
edge_index = add_self_loops(edge_index)[0]


clf_1 = XGBClassifier( tree_method='gpu_hist', 
                           n_estimators=1100,
                           max_depth=2,
                    random_state=42,
                    eta=0.3,
                    reg_lambda=0.001,
                           min_child_weight = 1,
                           max_delta_step= 3,
                           sampling_method= "uniform")
   
clf_2 = XGBClassifier( tree_method='gpu_hist', 
                           n_estimators=900,
                           max_depth=2,
                       random_state=42,
                       reg_lambda=0.2953684210526316,
                       eta=0.2733333333333333,
                           min_child_weight = 2,
                           max_delta_step= 4,
                           sampling_method= "uniform",
                      subsample=0.5)

clf_3 = SVC(probability=True, C=100, kernel="linear", degree=1)

def user_function(kwargs):
    return  kwargs["updated_features"] + kwargs["summed_neighbors"]

user_functions = [user_function, user_function,  user_function]
clfs = [clf_1, clf_2, clf_3]
hops_list = [0, 3,  8]
attention_configs = [ None, 
                     {'inter_layer_normalize': True,
                     'use_pseudo_attention':True,
                     'cosine_eps':.01,
                     'dropout_attn': None},
                     {'inter_layer_normalize': True,
                     'use_pseudo_attention':True,
                     'cosine_eps':.01,
                     'dropout_attn': None}
                    ]

In [48]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp,STATUS_OK, SparkTrials
from sklearn.metrics import accuracy_score

booster_self = ["gbtree"]
n_estimators_self = [5, 50, 100,150, 200, 400, 500, 800, 1_000, 1_200]
max_depth_self = [None, 1, 2, 3, 4]
max_delta_step_self = [None, 1, 2, 3, 4]
min_child_weight_self = [None, 1, 2, 3, 4]

xb_choices = {
    'booster': booster_self,
    'n_estimators': n_estimators_self,
    'max_depth': max_depth_self,
    'max_delta_step': max_delta_step_self,
    'min_child_weight': min_child_weight_self,
}
space_xb = {
    **{key: hp.choice(key, value) for key, value in xb_choices.items()},
    'eta': hp.loguniform('eta', -3, -.3),
    'subsample': hp.uniform('subsample', 0.6, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0, 1),
    'alpha': hp.uniform('alpha', 0, 1),
}
svc_choices = {
    'kernel': ["linear"],
}
space_svc = {
   **{key: hp.choice(key, value) for key, value in svc_choices.items()},
    "C":hp.uniform("C",0.0, 1.0),
}

space_lr = {
    "C":hp.uniform("C",0.0, 1.0)
}
stores=[]
hops_list = [[0,0], [3,3], [8,8]]
for hops_list in hops_lists:
    store=dict({})
    clfs = [XGBClassifier,SVC]
    spaces= [space_xb, space_svc, space_lr]
    space_choices= [xb_choices, svc_choices, {}]
    hops_list = hops_list
    attention_configs = [ None, 
                         {'inter_layer_normalize': True,
                         'use_pseudo_attention':True,
                         'cosine_eps':.01,
                         'dropout_attn': None}]
    max_acc = 0
    best_hops = None
    best_user_function = None
    best_attention_config = None
    best_clf_params = None
    best_clf = None
    for clf_idx, clf in enumerate(clfs):
        space = spaces[clf_idx]
        framework = Framework(user_functions, 
                     hops_list=hops_list, ## to obtain best for local neighborhood
                     clfs=[],
                     gpu_idx=0,
                     handle_nan=0.0,
                    attention_configs=attention_configs)


        val_aggregated_feature_list = framework.get_features(X, edge_index,val)
        train_aggregated_feature_list = framework.get_features(X, edge_index,train)## this shoould be indepent of the length of the classifiers list

        for i in range(len(val_aggregated_feature_list)):
            print(clf)
            print(i)
            print(hops_list[i])
            X_val = val_aggregated_feature_list[i].cpu()
            X_train = train_aggregated_feature_list[i].cpu()
            def objective(params):
                model = clf(**params, random_state = 42)
                model.fit(X_train, y[train])
                y_pred = model.predict(X_val)
                score = accuracy_score(y[val], y_pred)
                return {'loss': -score, 'status': STATUS_OK}
            spark_trials = SparkTrials()
            best_params = fmin(objective, space, algo=tpe.suggest, max_evals=50, trials=spark_trials)

            for key,value in space_choices[clf_idx].items():
                best_params[key] = value[best_params[key]]

            best_model = clf(**best_params)
            best_model.fit(X_train, y[train])
            y_pred = best_model.predict(X_val)
            score = accuracy_score(y[val], y_pred)
            if score >= max_acc:
                max_acc = score
                best_hops = hops_list[i]
                best_user_function = user_functions[i]
                best_attention_config = attention_configs[i]
                best_clf_params = best_params
                best_clf = clf
    store["hops_list"] = hops_list
    store["max_acc"] = max_acc
    store["best_hops"] = best_hops
    store["best_user_function"] = best_user_function
    store["best_attention_config"] = best_attention_config
    store["best_clf_params"] = best_clf_params
    store["best_clf"] = best_clf
    stores.append(store)

Because the requested parallelism was None or a non-positive value, parallelism will be set to (64), which is Spark's default parallelism (64), or 1, whichever is greater. We recommend setting parallelism explicitly to a positive value because the total of Spark task slots is subject to cluster sizing.


<class 'xgboost.sklearn.XGBClassifier'>
0
0
  0%|                                                                           | 0/200 [00:00<?, ?trial/s, best loss=?]

/home/dwalke/miniconda3/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/dwalke/miniconda3/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/dwalke/miniconda3/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/dwalke/miniconda3/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A Nu

  1%|▌                                                             | 2/200 [00:13<21:45,  6.59s/trial, best loss: -0.52]

/home/dwalke/miniconda3/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/dwalke/miniconda3/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/dwalke/miniconda3/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/dwalke/miniconda3/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A Nu

...

 98%|██████████████████████████████████████████████████████████ | 197/200 [03:21<00:03,  1.20s/trial, best loss: -0.774]

/home/dwalke/miniconda3/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


100%|██████████████████████████████████████████████████████████▋| 199/200 [03:22<00:00,  1.09trial/s, best loss: -0.774]

100%|███████████████████████████████████████████████████████████| 200/200 [03:23<00:00,  1.02s/trial, best loss: -0.774]

Total Trials: 200: 200 succeeded, 0 failed, 0 cancelled.


In [196]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp,STATUS_OK, SparkTrials
from sklearn.metrics import accuracy_score

booster_self = ["gbtree"]
n_estimators_self = [50,75, 100,125,150,175, 200,250, 300, 400, 500, 800]
max_depth_self = [None, 1, 2, 3, 4]
max_delta_step_self = [None, 1, 2, 3, 4]
min_child_weight_self = [None, 1, 2, 3, 4]

xb_choices = {
    'booster': booster_self,
    'n_estimators': n_estimators_self,
    'max_depth': max_depth_self,
    'max_delta_step': max_delta_step_self,
    'min_child_weight': min_child_weight_self,
}
space_xb = {
    **{key: hp.choice(key, value) for key, value in xb_choices.items()},
    'eta': hp.loguniform('eta', -3, -.3),
    'subsample': hp.uniform('subsample', 0.6, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0.5, 1.0),
    'alpha': hp.uniform('alpha', 0, 1),
}
svc_choices = {
    'kernel': ["linear"],
}
space_svc = {
   **{key: hp.choice(key, value) for key, value in svc_choices.items()},
    "C":hp.uniform("C",0.4, 1.0),
}

stores=[]
hops_list = [0,3,8]
clf_list = [0,3,8]
store=dict({})
clfs = [XGBClassifier,XGBClassifier,SVC]
spaces= [space_xb,space_xb, space_svc]
space_choices= [xb_choices,xb_choices, svc_choices]
hops_list = hops_list
attention_configs = [ {'inter_layer_normalize': True,
                     'use_pseudo_attention':True,
                     'cosine_eps':.01,
                     'dropout_attn': None},{'inter_layer_normalize': True,
                     'use_pseudo_attention':True,
                     'cosine_eps':.01,
                     'dropout_attn': None},{'inter_layer_normalize': True,
                     'use_pseudo_attention':True,
                     'cosine_eps':.01,
                     'dropout_attn': None}]
user_functions = [user_function, user_function, user_function]
max_acc = 0
best_hops = None
best_user_function = None
best_attention_config = None
best_clf_params = None
best_clf = None
    
framework = Framework(user_functions, 
             hops_list=hops_list, ## to obtain best for local neighborhood
             clfs=[],
             gpu_idx=0,
             handle_nan=0.0,
            attention_configs=attention_configs)

val_aggregated_feature_list = framework.get_features(X, edge_index,val)
train_aggregated_feature_list = framework.get_features(X, edge_index,train)## this shoould be indepent of the length of the classifiers list

for i in range(len(val_aggregated_feature_list)):
    store=dict({})
    print(i)
    print( clfs[i])
    X_val = val_aggregated_feature_list[i].cpu()
    X_train = train_aggregated_feature_list[i].cpu()
    def objective(params):
        model = clfs[i](**params, random_state = 42) if i>=2 else clfs[i](**params, random_state = 42, tree_method = "hist", device = "cuda")
        model.fit(X_train, y[train])
        y_pred = model.predict(X_val)
        score = accuracy_score(y[val], y_pred)
        return {'loss': -score, 'status': STATUS_OK}
    spark_trials = SparkTrials()
    best_params = fmin(objective, spaces[i], algo=tpe.suggest, max_evals=200, trials=spark_trials)

    for key,value in space_choices[i].items():
        best_params[key] = value[best_params[key]]

    best_model = clfs[i](**best_params)
    best_model.fit(X_train, y[train])
    y_pred = best_model.predict(X_val)
#     score = accuracy_score(y[val], y_pred)
#     if score >= max_acc:
    max_acc = score
    best_hops = hops_list[i]
    best_user_function = user_functions[i]
    best_attention_config = attention_configs[i]
    best_clf_params = best_params
    best_clf = clfs[i]
    store["hops_list"] = hops_list
    store["max_acc"] = max_acc
    store["best_hops"] = best_hops
    store["best_user_function"] = best_user_function
    store["best_attention_config"] = best_attention_config
    store["best_clf_params"] = best_clf_params
    store["best_clf"] = best_clf
    stores.append(store)

Because the requested parallelism was None or a non-positive value, parallelism will be set to (64), which is Spark's default parallelism (64), or 1, whichever is greater. We recommend setting parallelism explicitly to a positive value because the total of Spark task slots is subject to cluster sizing.


0
<class 'xgboost.sklearn.XGBClassifier'>
  0%|                                                                           | 0/200 [00:00<?, ?trial/s, best loss=?]

/home/dwalke/miniconda3/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/dwalke/miniconda3/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/dwalke/miniconda3/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/dwalke/miniconda3/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A Nu

  0%|▎                                                            | 1/200 [00:07<23:22,  7.05s/trial, best loss: -0.466]

/home/dwalke/miniconda3/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/dwalke/miniconda3/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


...

 98%|██████████████████████████████████████████████████████████ | 197/200 [03:21<00:03,  1.01s/trial, best loss: -0.774]

/home/dwalke/miniconda3/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


 99%|██████████████████████████████████████████████████████████▍| 198/200 [03:22<00:02,  1.01s/trial, best loss: -0.774]

100%|███████████████████████████████████████████████████████████| 200/200 [03:23<00:00,  1.02s/trial, best loss: -0.774]

Total Trials: 200: 200 succeeded, 0 failed, 0 cancelled.


In [197]:
stores

[{'hops_list': [0, 3, 8],
  'max_acc': 0.774,
  'best_hops': 0,
  'best_user_function': <function __main__.user_function(kwargs)>,
  'best_attention_config': {'inter_layer_normalize': True,
   'use_pseudo_attention': True,
   'cosine_eps': 0.01,
   'dropout_attn': None},
  'best_clf_params': {'alpha': 0.7419628127619498,
   'booster': 'gbtree',
   'eta': 0.051585868540301553,
   'max_delta_step': 1,
   'max_depth': 2,
   'min_child_weight': None,
   'n_estimators': 125,
   'reg_lambda': 0.8735979022497954,
   'subsample': 0.9642147572324742},
  'best_clf': xgboost.sklearn.XGBClassifier},
 {'hops_list': [0, 3, 8],
  'max_acc': 0.774,
  'best_hops': 3,
  'best_user_function': <function __main__.user_function(kwargs)>,
  'best_attention_config': {'inter_layer_normalize': True,
   'use_pseudo_attention': True,
   'cosine_eps': 0.01,
   'dropout_attn': None},
  'best_clf_params': {'alpha': 0.76429215096084,
   'booster': 'gbtree',
   'eta': 0.15896917501655425,
   'max_delta_step': 1,
   'm

In [206]:
attention_configs = list(map(lambda s: s["best_attention_config"], stores))
clfs = list(map(lambda s: s["best_clf"], stores))
print(clfs)
hops_list = [0,3,8]
user_functions = [user_function, user_function, user_function]
models = []
for i, clf in enumerate(clfs):
    model = clf(**stores[i]["best_clf_params"], probability=True) if i == 2 else clf(**stores[i]["best_clf_params"], device="cuda:0", eval_metric="mlogloss")
    models.append(model)

[<class 'xgboost.sklearn.XGBClassifier'>, <class 'xgboost.sklearn.XGBClassifier'>, <class 'sklearn.svm._classes.SVC'>]


In [207]:
# models = models[::-1]
## ppartial dep

In [208]:
import time
start = time.time()
framework = Framework(user_functions, 
                     hops_list=hops_list, ## to obtain best for local neighborhood
                     clfs=models,
                     gpu_idx=0,
                     handle_nan=0.0,
                    attention_configs=attention_configs)

val_0, val_3, val_8 = framework.get_features(X, edge_index,val)
val_0, val_3, val_8 = val_0.cpu(), val_3.cpu(), val_8.cpu()
kwargs_list=[{"eval_set":[(val_0, y[val])], "early_stopping_rounds":5}, {"eval_set":[(val_3, y[val])], "early_stopping_rounds":5}, {}]
framework.fit(X, edge_index, y, train, kwargs_list)
print(time.time()-start)

[10:59:42] WARNING: ../src/learner.cc:767: 
Parameters: { "device" } are not used.

[0]	validation_0-mlogloss:1.92944
[1]	validation_0-mlogloss:1.91221
[2]	validation_0-mlogloss:1.89614
[3]	validation_0-mlogloss:1.87916
[4]	validation_0-mlogloss:1.86238
[5]	validation_0-mlogloss:1.84455
[6]	validation_0-mlogloss:1.82877
[7]	validation_0-mlogloss:1.81218
[8]	validation_0-mlogloss:1.79689
[9]	validation_0-mlogloss:1.78102
[10]	validation_0-mlogloss:1.76661
[11]	validation_0-mlogloss:1.75246
[12]	validation_0-mlogloss:1.73772
[13]	validation_0-mlogloss:1.72252
[14]	validation_0-mlogloss:1.70760
[15]	validation_0-mlogloss:1.69377
[16]	validation_0-mlogloss:1.67997
[17]	validation_0-mlogloss:1.66659
[18]	validation_0-mlogloss:1.65424
[19]	validation_0-mlogloss:1.64274
[20]	validation_0-mlogloss:1.63107
[21]	validation_0-mlogloss:1.62010
[22]	validation_0-mlogloss:1.60978
[23]	validation_0-mlogloss:1.59845
[24]	validation_0-mlogloss:1.58914
[25]	validation_0-mlogloss:1.57985
[26]	validation_

In [209]:
import numpy as np
from sklearn.metrics import accuracy_score
pred = framework.predict(X, edge_index, test) 
pred_val = framework.predict(X, edge_index, val) 
y_test = y[test]
y_val = y[val]
print(accuracy_score(y_val, pred_val))
print(accuracy_score(y_test, pred))

0.8
0.82


In [163]:
os.getcwd()

'/home/dwalke/git/graph_aware_ml/models/825'

In [164]:
# import pickle
# import os

# os.chdir("../../models/827")
# for i, clf in enumerate(framework.clfs):
#     file_name = f"{hops_list[i]}_{clf.__class__.__name__}.pkl"
#     with open(file_name, 'wb') as file:
#         pickle.dump(clf, file)

In [196]:
def print_best():
    print(max_acc)
    print(best_hops)
    print(best_user_function)
    print(best_attention_config)
    print(best_clf_params)
    print(best_clf)
print_best()

0.808
8
<function user_function at 0x7f3e6c1ddc60>
{'inter_layer_normalize': True, 'use_pseudo_attention': True, 'cosine_eps': 0.01, 'dropout_attn': None}
{'alpha': 0.02045099591792793, 'booster': 'dart', 'colsample_bylevel': 0.9043890199689195, 'colsample_bynode': 0.9610703768457423, 'colsample_bytree': 0.8011914765143247, 'eta': 0.0076757402952727215, 'max_delta_step': 2, 'max_depth': 3, 'min_child_weight': 2, 'n_estimators': 90, 'rate_drop': 0.7646425507490928, 'reg_lambda': 0.0939497647035095, 'skip_drop': 0.6882676580589697, 'subsample': 0.7359911712096356}
<class 'xgboost.sklearn.XGBClassifier'>


In [14]:
clf_0 = XGBClassifier(**{'alpha': 0.020563956192073844, 'booster': 'dart', 'colsample_bylevel': 0.8128692466913029, 'colsample_bynode': 0.9910960381130457, 'colsample_bytree': 0.8367129518356438, 'eta': 0.01286147687208788, 'max_delta_step': None, 'max_depth': 2, 'min_child_weight': 1, 'n_estimators': 400, 'rate_drop': 0.17911622702128605, 'reg_lambda': 0.09721148774327494, 'skip_drop': 0.6763043153026116, 'subsample': 0.5526163189757906}, tree_method='gpu_hist', early_stopping_rounds = 3, eval_metric="mlogloss")
attn_0 = {'inter_layer_normalize': True, 'use_pseudo_attention': True, 'cosine_eps': 0.01, 'dropout_attn': None}
hops_0 = 0

clf_3 = XGBClassifier(**{'alpha': 0.09214023868921617, 'booster': 'dart', 'colsample_bylevel': 0.7948477467254295, 'colsample_bynode': 0.757122015098133, 'colsample_bytree': 0.9174833944885485, 'eta': 0.14972740962118347, 'max_delta_step': 2, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 120, 'rate_drop': 0.9772605030384062, 'reg_lambda': 0.09072783671555298, 'skip_drop': 0.975347237870791, 'subsample': 0.5770667775984667},tree_method='gpu_hist',early_stopping_rounds = 3, eval_metric="mlogloss")
attn_3 = {'inter_layer_normalize': True, 'use_pseudo_attention': True, 'cosine_eps': 0.01, 'dropout_attn': None}
hops_3 = 3

# clf_8 = XGBClassifier(**{'alpha': 0.02045099591792793, 'booster': 'dart', 'colsample_bylevel': 0.9043890199689195, 'colsample_bynode': 0.9610703768457423, 'colsample_bytree': 0.8011914765143247, 'eta': 0.0076757402952727215, 'max_delta_step': 2, 'max_depth': 3, 'min_child_weight': 2, 'n_estimators': 90, 'rate_drop': 0.7646425507490928, 'reg_lambda': 0.0939497647035095, 'skip_drop': 0.6882676580589697, 'subsample': 0.7359911712096356})
clf_8 = SVC(probability=True, C=100, kernel="linear", degree=1)
attn_8 = {'inter_layer_normalize': True, 'use_pseudo_attention': True, 'cosine_eps': 0.01, 'dropout_attn': None}
hops_8 = 8

In [15]:
user_functions = [user_function, user_function,  user_function]
clfs = [clf_0, clf_3, clf_8]
hops_list = [hops_0, hops_3,  hops_8]
attention_configs = [ attn_0, 
                     attn_3,
                     attn_8
                    ]

In [18]:
framework = Framework(user_functions, 
                 hops_list,
                 clfs,
                 gpu_idx=0,
                 handle_nan=0.0,
                attention_configs=attention_configs)

self_val, neighbors_val, neighbors_val_2  = framework.get_features(X, edge_index,val)
self_val, neighbors_val, neighbors_val_2 = self_val.cpu(), neighbors_val.cpu(), neighbors_val_2.cpu()
kwargs_list=[{
    "eval_set":[(self_val, y[val])]
}, {"eval_set":[(neighbors_val, y[val])]}, {}]
framework.fit(X, edge_index, y, train, kwargs_list=kwargs_list)

[0]	validation_0-mlogloss:1.93875
[1]	validation_0-mlogloss:1.93353
[2]	validation_0-mlogloss:1.93187
[3]	validation_0-mlogloss:1.92315
[4]	validation_0-mlogloss:1.91402
[5]	validation_0-mlogloss:1.90630
[6]	validation_0-mlogloss:1.90575
[7]	validation_0-mlogloss:1.89769
[8]	validation_0-mlogloss:1.89645
[9]	validation_0-mlogloss:1.88854
[10]	validation_0-mlogloss:1.88087
[11]	validation_0-mlogloss:1.87311
[12]	validation_0-mlogloss:1.86576
[13]	validation_0-mlogloss:1.86202
[14]	validation_0-mlogloss:1.85702
[15]	validation_0-mlogloss:1.85087
[16]	validation_0-mlogloss:1.84601
[17]	validation_0-mlogloss:1.84049
[18]	validation_0-mlogloss:1.84023
[19]	validation_0-mlogloss:1.83360
[20]	validation_0-mlogloss:1.82620
[21]	validation_0-mlogloss:1.82108
[22]	validation_0-mlogloss:1.81470
[23]	validation_0-mlogloss:1.81453
[24]	validation_0-mlogloss:1.80792
[25]	validation_0-mlogloss:1.80283
[26]	validation_0-mlogloss:1.79763
[27]	validation_0-mlogloss:1.79746
[28]	validation_0-mlogloss:1.7

[231]	validation_0-mlogloss:1.44858
[232]	validation_0-mlogloss:1.44719
[233]	validation_0-mlogloss:1.44600
[234]	validation_0-mlogloss:1.44600
[235]	validation_0-mlogloss:1.44392
[236]	validation_0-mlogloss:1.44272
[237]	validation_0-mlogloss:1.44271
[238]	validation_0-mlogloss:1.44271
[239]	validation_0-mlogloss:1.44270
[240]	validation_0-mlogloss:1.44111
[241]	validation_0-mlogloss:1.44063
[242]	validation_0-mlogloss:1.44062
[243]	validation_0-mlogloss:1.43993
[244]	validation_0-mlogloss:1.43879
[245]	validation_0-mlogloss:1.43779
[246]	validation_0-mlogloss:1.43779
[247]	validation_0-mlogloss:1.43778
[248]	validation_0-mlogloss:1.43534
[249]	validation_0-mlogloss:1.43520
[250]	validation_0-mlogloss:1.43519
[251]	validation_0-mlogloss:1.43519
[252]	validation_0-mlogloss:1.43444
[253]	validation_0-mlogloss:1.43255
[254]	validation_0-mlogloss:1.43255
[255]	validation_0-mlogloss:1.43064
[256]	validation_0-mlogloss:1.43000
[257]	validation_0-mlogloss:1.42930
[258]	validation_0-mlogloss:

[XGBClassifier(alpha=0.020563956192073844, base_score=None, booster='dart',
               callbacks=None, colsample_bylevel=0.8128692466913029,
               colsample_bynode=0.9910960381130457,
               colsample_bytree=0.8367129518356438, early_stopping_rounds=3,
               enable_categorical=False, eta=0.01286147687208788,
               eval_metric='mlogloss', feature_types=None, gamma=None,
               gpu_id=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=None, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=2, max_leaves=None,
               min_child_weight=1, missing=nan, monotone_constraints=None,
               n_estimators=400, n_jobs=None, num_parallel_tree=None, ...),
 XGBClassifier(alpha=0.09214023868921617, base_score=None, booster='dart',
               callbacks=None, colsample_bylevel=0.7948477467254295,
               col

In [19]:
import numpy as np
from sklearn.metrics import accuracy_score
pred = framework.predict(X, edge_index, test) 
pred_val = framework.predict(X, edge_index, val) 
y_test = y[test]
y_val = y[val]
print(accuracy_score(y_val, pred_val))
print(accuracy_score(y_test, pred))

0.786
0.81


In [210]:
from tqdm.notebook import tqdm
max_val = 0
max_test = 0
best_weights = None
for weight_0 in tqdm(np.linspace(0,1, 5)):
    for weight_1 in tqdm(np.linspace(0,1,5)):
        pred = framework.predict(X, edge_index, test, weights=[weight_0, weight_1, 1-weight_0-weight_1]) 
        pred_val = framework.predict(X, edge_index, val, weights=[weight_0, weight_1, 1-weight_0-weight_1]) 
        y_test = y[test]
        y_val = y[val]
        acc_val = accuracy_score(y_val, pred_val)
        acc_test = accuracy_score(y_test, pred)
        
        if acc_val >= max_val:
            max_val = acc_val
            max_test = acc_test
            best_weights = [weight_0, weight_1, 1-weight_0-weight_1]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [211]:
print(max_val)
print(max_test)
print(best_weights)

0.814
0.826
[0.25, 0.0, 0.75]
